# SDialog dependencies

In [ ]:
%%script false --no-raise-error
# Only run this cell if you are using a non jupyter environment
!conda create --name jsalt python=3.9 -y
!conda activate jsalt
!conda install sox
!pip install -r sdialog/requirements.txt
!pip install -r sdialog/requirements-audio.txt

In [ ]:
import os
import json
from tqdm import tqdm

In [ ]:
import sdialog
from sdialog import Dialog
from sdialog.generators import PersonaGenerator
from sdialog.personas import Persona, PersonaAgent, Doctor, Patient, Agent

# Generate Persona

In [ ]:
sdialog.config.set_llm("aws:anthropic.claude-3-5-sonnet-20240620-v1:0", region_name="us-east-1")

In [ ]:
%%script false --no-raise-error
persona_card_folder = "./personas"

# Generate doctor persona
doctor_persona = Doctor(
    name="Dr. Smith",
    gender="male",
    age=52,
    specialty="Family Medicine"
)
generator_doctor = PersonaGenerator(doctor_persona)
persona_cards = generator_doctor.generate(n=1)
persona_cards.to_file(f"{persona_card_folder}/persona_doctor.json")

# Generate patient persona
patient_persona = Patient(
    name="John Doe",
    gender="male",
    age=62
)
generator_patient = PersonaGenerator(patient_persona)
persona_cards = generator_patient.generate(n=1)
persona_cards.to_file(f"{persona_card_folder}/persona_patient.json")


# Load persona

In [ ]:
# Load personas
persona_doctor = Persona.from_file("./personas/persona_doctor.json")
persona_patient = Persona.from_file("./personas/persona_patient.json")

In [ ]:
%%script false --no-raise-error
context = "Generate me a 50 turn medical dialogue between patient and doctor, for a primary care visit"

# Create agents
agent1 = PersonaAgent(persona=persona_doctor, name="DOCTOR", dialogue_details=context, response_details="make short turn answers when needed")
agent2 = PersonaAgent(persona=persona_patient, name="PATIENT", dialogue_details=context, response_details="make short turn answers when needed")

In [ ]:
save_all = True
GENERATE_PERSONA = True
FORCE_DIALOG_GENERATION = False

os.makedirs("./outputs", exist_ok=True)

In [ ]:
if FORCE_DIALOG_GENERATION:

    dialog = agent1.talk_with(agent2, max_turns=3)
    dialog.to_file("dialog_demo.json")

else:
    dialog = Dialog.from_file("dialog_demo.json")

dialog.print()

# Audio dependencies

# Instantiating voices database

In [ ]:
from sdialog.audio.voice_database import DummyVoiceDatabase

In [ ]:
dummy_voice_database = DummyVoiceDatabase()
print("Instantiating voice database...")

# Instantiating TTS pipeline

In [ ]:
from sdialog.audio.tts_engine import KokoroTTS # ChatterboxTTS, XttsTTS

In [ ]:
tts_pipeline = KokoroTTS()
# tts_pipeline = ChatterboxTTS()
# tts_pipeline = XttsTTS()
print("Instantiating TTS pipeline...")

In [ ]:
from sdialog.audio.audio_dialog import AudioDialog

In [ ]:
# Extend the turns with audio attributes
dialog: AudioDialog = AudioDialog.from_dialog(dialog)

In [ ]:
%%script false --no-raise-error
!git clone https://github.com/cyrta/dscaper.git

In [ ]:
%%script false --no-raise-error
!pip install -e ./dscaper

In [ ]:
import os
import scaper
DATA_PATH = "./dscaper_data"
os.makedirs(DATA_PATH, exist_ok=True)
dsc = scaper.Dscaper(dscaper_base_path=DATA_PATH)

In [ ]:
from sdialog.audio.audio_pipeline import AudioPipeline

In [ ]:
from sdialog.audio.audio_events_enricher import AudioEventsEnricher
enricher = AudioEventsEnricher()

In [ ]:
audio_pipeline = AudioPipeline(
    voice_database=dummy_voice_database,
    tts_pipeline=tts_pipeline,
    dir_audio="./outputs",
    dscaper=dsc,
    enricher = enricher
)

In [ ]:
audio_pipeline.populate_dscaper(["sdialog/background","sdialog/foreground"])

In [ ]:
from sdialog.audio.room import MicrophonePosition
from sdialog.audio.room_generator import RoomGenerator, RoomRole

In [ ]:
room = RoomGenerator().generate(RoomRole.CONSULTATION)
print(room)

In [ ]:
dialog: AudioDialog = audio_pipeline.inference(
    dialog,
    microphone_position=MicrophonePosition.MONITOR,
    room=room,
    do_word_alignments=False,
    do_snr=False,
    do_room_position=False
)

In [ ]:
for turn in dialog.turns:
    print(turn.text)
    print(turn.position)
    print(turn.snr)
    print("____________________")

## dialog.to_audio()

In [ ]:
# TODO: Add a demo of the audio generation pipeline from the dialog object

# Audio Evaluation

## Utterance level evaluation

In [ ]:
from sdialog.audio.evaluation import compute_evaluation_utterances, compute_evaluation_audio

In [ ]:
# Utterances level evaluation
metrics_utterances_level = compute_evaluation_utterances(dialog)
for key, value in metrics_utterances_level.items():
    print(f"{key}: {value}")

# Audio level evaluation
metrics_audio_level = compute_evaluation_audio(dialog)
for key, value in metrics_audio_level.items():
    print(f"{key}: {value}")

## Audio level evaluation

In [ ]:
# TODO: Evaluate the final generated audio